In [1]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
from math import floor
import pickle
from pymongo import MongoClient
from os import listdir
from os.path import isfile, join, isdir
from os.path import isfile, join, isdir
from os import listdir
import re
import pickle
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
class dbConf(object):
    # connect to database
    client = MongoClient('mongodb://localhost:27017/')
    try:
        mdb = client.racmo
        Notifications = mdb.Notifications
        PdfFiles=mdb.PdfFiles
        Keywords=mdb.Keywords
    except Exception as e:
        print(e)

PDF_DIR = "/home/thrymr/Racmo/processed/Final version of Notifications/"

In [55]:
KeywordDir="/home/thrymr/Racmo/Notification Keywords/"
keyword_files = [join(KeywordDir,f) for f in listdir(KeywordDir)\
                 if isfile(join(KeywordDir, f)) and 
                 f[-4:].lower() == "xlsx"]
ls=[]

for f in keyword_files:
    xl_file = pd.ExcelFile(f)
    dfs = xl_file.parse(xl_file.sheet_names[0])

    tic=False
    for index, row in dfs.iterrows():
        ite=dict()
        if(row[0]=='TICKETS' or row[0]=='TICKET'):
            tic=True
        if(row[0]!='TICKETS' and row[0]!='TICKET' and isinstance(row[0], basestring)):

            ite['file_class']=f.split('/')[-1].split()[0]
            ite['notification_type']=row[0]
            if(tic):
                ite['file_type']='TICKET'
            else:
                ite['file_type']='NOTIFICATION'
            kw=list()
            for i in range(1,row.size):
                if(isinstance(row[i], basestring) and row[i]!='+'):
                    kw.append(unidecode.unidecode(row[i]).lower() )
            ite['keyword']=kw        
            ls.append(ite)
newkdf=pd.DataFrame(ls)
# newkdf

In [3]:
with open('train.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [4]:
# kdf=b['keywords']
kdf=newkdf
fdf=b['documents']

In [49]:
# with open('train2.pickle', 'rb') as handle:
#     b = pickle.load(handle)
#     df=b['documents']
    
# fdf['table_response']=df['table_response']
# fdf

In [6]:
pdfclass={'N1':'ADMISSION','N2':'TRANSFER OF LEGAL REPRESENTATION PERMITTED',
           'N3':'ENFORCEMENT ORDERS','N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS','N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING','N8':'ASSET INQUIRY',
           'N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT'}

In [7]:
kdf['purpose']=''
n=kdf.iterrows()
for i,row in n:
    if 'EJECUCION DE TITULOS NO JUDICIALES'.lower() in row['keyword']:
        kdf=kdf.drop(kdf.index[i])
    elif(not ( row['file_class'] =='N5'or row['file_class'] =='N6')):
        if(pdfclass[row['file_class']] in row['notification_type']):
            
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N5':
        if('N5 - REJECTED CLAIMS' in row['notification_type']):
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N6':
        if('N6 - REJECTED TRANSFER OF LEGAL REPRESENTATION' in row['notification_type']):
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'


In [50]:
# k=kdf[(kdf['file_class']=='N10') & (kdf['purpose']=='CLASSIFICATION')&(kdf['file_type']=='NOTIFICATION')].groupby('notification_type')
# print(k.groups)
# new10df=pd.DataFrame(columns=kdf.columns)
# i=0
# for lidx in k.groups['N10 - REQUIREMENTS']:
#     for ridx in k.groups['N10 -  REQUIREMENT SPECIFICATION']:
#         new10df=new10df.append(kdf.iloc[lidx],ignore_index=True)
        
#         new10df.loc[i,'keyword']=new10df.loc[i,'keyword']+kdf.loc[ridx,'keyword']
#         i=i+1

# new10df

In [51]:
# fdf=b['documents']
# fdf['fileclass']=fdf['fileclass'].apply(lambda x :x.split()[0])
# fdf

In [10]:
kdf=kdf[~((kdf['file_class']=='N10') & (kdf['purpose']=='CLASSIFICATION')&(kdf['file_type']=='NOTIFICATION'))].append(new10df,ignore_index=True)
# kdf

In [11]:
kdf=kdf.rename(columns={'file_class':'fileclass','file_type':'filetype'})


In [12]:
for i,r in fdf.iterrows():
    try:
        newtext =unicode(textract.process(join(PDF_DIR,r['filename'])),'utf-8')
        newtext=(''.join((c for c in unicodedata.normalize('NFD', newtext) if unicodedata.category(c) != 'Mn'))).lower()
        rem=''
        paratlist=['MODO DE IMPUGNACION:'.lower(),'mode d\'impugnacio','recurso de repelacion','recurs de reposicio','recurso de reposicion','recurso de apelacion']
        if (r['filetype']=='NOTIFICATION') :
            for parat in paratlist:
                if (parat.lower() in newtext) :
                    rem=re.split(r'\s(?=(?:y firmo|y firma))',newtext.split(parat.lower())[-1],1)[0]
            newtext=newtext.replace(rem,'')
    except Exception as e:
        newtext='Error:'+str(e)
    fdf.loc[i,'text_response']=newtext

In [52]:
# import re
# kdf["document_list"]=np.empty((len(kdf), 0)).tolist()
# fdf["keywords"]=fdf["hasid"].apply(lambda x:{})

# notextset=set()
# for i,kdrow in kdf.iterrows():
#     for j,row in fdf.iterrows():
#         text=''.join(row['text_response'].split())
# #         
#         if row['text_response'][:5]!='Error':
            
#             f=True
#             for k in kdrow['keyword']:
# #                 if not re.search(k, row['text_response'], re.IGNORECASE):
                
# #                 print(unidecode.unidecode(k).lower(),find_near_matches(unidecode.unidecode(k).lower(), text, max_l_dist=1))
# #                 match=[]
# #                 try:
# #                     match=find_near_matches(str(unidecode.unidecode(k)).lower(), text, max_l_dist=2)
# #                 except Exception as e:
# #                     notextset.add(j)
# #                 if len(match)==0:
# #                     f=False
#                 if not (''.join(unidecode.unidecode(unicode(k,'utf-8')).split()).lower() in text):
#                     f=False
#             if f and (kdrow['filetype']==row['filetype'])and (kdrow['purpose']=='CLASSIFICATION'):
#                 kdf.loc[i,"document_list"].append(row['hasid'])
                
#                 if kdrow['fileclass'] in fdf.loc[j,'keywords'].keys():
#                     fdf.loc[j,'keywords'][kdrow['fileclass']].append(kdrow['keyword'])
#                 else:
#                     fdf.loc[j,'keywords'][kdrow['fileclass']]=list()
#                     fdf.loc[j,'keywords'][kdrow['fileclass']].append(kdrow['keyword'])
# fdf
# # for idf,rowdf in fdf.iterrows():
# #     keywords=[]
# #     keywordclass=[]
    
# #     for ki, kr in kddf.iterrows():
# #         if rowdf['hasid'] in kr['document_list'] :
# #             if kr['file_class'] in fdf.columns:
# #                 dokydf.loc[idf,kr['file_class']]+=1
# #             else:
# #                 dokydf[kr['file_class']]=0
# #                 dokydf.loc[idf,kr['file_class']]=1
# # .to_csv("keywords.csv")  
# # keyrpo=pd.concat([keyrpo.drop(['key_count'], axis=1), keyrpo['key_count'].apply(pd.Series)], axis=1)


In [14]:
# fdf['fileclass']=fdf['filename'].apply(lambda x : x.split()[0])
# kdf.to_csv('dist2_keyword_train.csv')
a = {'documents':fdf,'keywords':kdf}
with open('Modified_train.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
import textract

_kdf = kdf[kdf.document_list.apply(lambda x : len(x) == 0)]
_kdf[(kdf['fileclass']=='N1')&(kdf['purpose']=='CLASSIFICATION')]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,fileclass,filetype,keyword,notification_type,purpose,document_list
41,N1,TICKET,[notifica admisa'n y requerimiento de pago],N1 - ADMISSION,CLASSIFICATION,[]
42,N1,TICKET,[noti. admissia requeriment de pagament],N1 - ADMISSION,CLASSIFICATION,[]


In [16]:

with open('Modified_train.pickle', 'rb') as handle:
    b=pickle.load(handle)
kdf=b['keywords']
fdf=b['documents']   

In [17]:
dokydf=fdf.copy()[['filename','filetype','keywords']]
keyrpodf=kdf.copy()
for idf,rowdf in fdf.iterrows():
    keywords=[]
    keywordclass=[]
    for k,v in rowdf['keywords'].items():
        if 'key '+k in dokydf.columns:
                dokydf.loc[idf,'key '+k].append(v)
        else:
                dokydf['key '+k]=np.empty((len(dokydf), 0)).tolist()
                dokydf.loc[idf,'key '+k].append(v)
    for ki, kr in kdf.iterrows():
            if rowdf['hasid'] in kr['document_list'] :
                if kr['fileclass'] in dokydf.columns:
                    dokydf.loc[idf,kr['fileclass']]+=1
                else:
                    dokydf[kr['fileclass']]=0
                    dokydf.loc[idf,kr['fileclass']]=1
                if 'Filename '+ rowdf['fileclass'] in keyrpodf.columns:
                    keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])
                else:
                    keyrpodf['Filename '+ rowdf['fileclass']]=np.empty((len(keyrpodf), 0)).tolist()
                    keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])
        

# .to_csv("keywords.csv")  
# keyrpo=pd.concat([keyrpo.drop(['key_count'], axis=1), keyrpo['key_count'].apply(pd.Series)], axis=1)

In [18]:
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/keywordDetails.xlsx')
keyrpodf.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/FileDetails.xlsx')
dokydf.to_excel(writer,'Sheet1')
writer.save()

In [20]:
kxddf=keyrpodf.copy()[['fileclass','filetype','purpose','notification_type','keyword','document_list']]
col=[x.split()[-1] for x in keyrpodf.columns[-10:] ]
for i,r in keyrpodf.iterrows():
    for c in col:
        kxddf.loc[i,c]=len(keyrpodf.loc[i,'Filename '+c])


In [54]:
for i,row in kxddf.iterrows():
    biased=''
    keyscore=dict(row[-10:])
    biased=max(keyscore, key=keyscore.get)
    if keyscore[biased]>0:
        kxddf.loc[i,'bias']=biased
    else:
        kxddf.loc[i,'bias']=row['fileclass']


In [22]:

a = {'keywordsX':kxddf[['fileclass','filetype','purpose','notification_type','keyword','bias']]}
with open('Final_Keyword_Analysis.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
a = {'keywordsX':kxddf}
with open('Keyword_Analysis.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
y_actu=pd.Series(list(ddf['fileclass']), name='Actual')

y_pred = pd.Series(list(ddf['predicted_class']), name='Predicted')
y_pred=y_pred.apply(lambda x:'_alpha' if x!=x else x)
df_confusion = pd.crosstab(y_actu, y_pred,margins=True)

In [26]:
df_confusion


Predicted,N1,N10,N2,N2+N4,N3,N4,N5,N6,N7,N8,N9,_alpha,All
Actual,,,,,,,,,,,,,
N1,41,0,0,0,4,0,0,0,0,0,1,7,53
N10,0,24,1,0,0,0,2,2,0,0,0,29,58
N2,0,0,40,2,0,0,0,0,0,0,0,16,58
N3,0,0,0,0,44,0,0,0,0,0,0,13,57
N4,0,0,0,0,0,50,0,0,0,0,0,8,58
N5,0,1,0,0,0,0,35,1,0,0,0,22,59
N6,0,0,1,0,0,0,1,27,0,0,0,29,58
N7,0,0,0,0,0,0,0,0,46,0,0,10,56
N8,0,0,0,0,0,0,0,0,0,29,0,10,39


In [27]:
fgs=ddf.groupby('filegroup')
len(fgs.groups)

293

In [58]:
fgdf=pd.DataFrame(columns=['filegroup','filegroupclass'])
i=0
fgdf['files']=np.empty((293, 0)).tolist()
fgdf['filetypes']=np.empty((293, 0)).tolist()
fgdf['predicted_classes']=np.empty((293, 0)).tolist()
for k,v in fgs.groups.items():
    
    fgdf.loc[i,'filegroup']=k
    fgdf.loc[i,'filegroupclass']=ddf.loc[v[0],'fileclass']
    files=[]
    pcs=[]
    filetypes=[]
    for ind in v:
            files.append(ddf.loc[ind,'filename'])
            pcs.append(ddf.loc[ind,'predicted_class'])
            filetypes.append(ddf.loc[ind,'filetype'])
    fgdf.loc[i,'filetypes']=filetypes
    fgdf.loc[i,'files']=files
    fgdf.loc[i,'predicted_classes']=pcs
    i+=1
fgdf          

In [29]:
y_actufg=[]
y_predfg=[]
conflic=[]
for k,v in fgs.groups.items():
    y_actufg.append(ddf.loc[v[0],'fileclass'])
    s=set()
    e='_alpha'
    for i in v:
        s.add(ddf.loc[i,'predicted_class'])
#     s={x for x in s if x==x}
    if(len(s)==1):
        if list(s)[0]==list(s)[0]:
            e=list(s)[0]
            y_predfg.append(list(s)[0])
    else:
        
        if(len({x for x in s if x==x})>1):
            e='Conflict'
             
        else:
            
            for x in list(s):
                if x==x:
                    e=x
    if e!='_alpha' and e!='Conflict':
        fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
    elif e=='Conflict':
        for j in v:
            if ddf.loc[j,'filetype']=='TICKET':
                e=ddf.loc[j,'predicted_class']
                fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
    y_predfg.append(e)        
        
        

In [30]:
fgdf[~(fgdf['filegroupclass']==fgdf['group_predicted_class'])&(fgdf['filegroupclass']=='N6')]

,filegroup,filegroupclass,files,filetypes,predicted_classes,group_predicted_class
16,01INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[TICKET, NOTIFICATION]","[nan, nan]",NaN
29,29INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,[NOTIFICATION],[nan],NaN
65,20INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[NOTIFICATION, TICKET]","[nan, nan]",NaN
113,12INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[TICKET, NOTIFICATION]","[nan, nan]",NaN
125,18INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[TICKET, NOTIFICATION]","[nan, N5]",N5
253,08INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[TICKET, NOTIFICATION]","[N2, nan]",N2
278,21INADMITIDASUCESIONPROCESAL,N6,[N6 REJECTED TRANSFER OF LEGAL REPRESENTATION/...,"[TICKET, NOTIFICATION]","[nan, nan]",NaN


In [31]:
a = {'documents':fdf,'keywords':kdf,'filegroup':fgdf}
with open('Modified_train1.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
df_confusionfg = pd.crosstab(pd.Series(list(fgdf['filegroupclass']),name='Actual'), pd.Series(list(fgdf['group_predicted_class'].fillna('_alpha')),name='Predicted'),margins=True)
df_confusionfg

Predicted,N1,N10,N2,N2+N4,N3,N4,N5,N6,N7,N8,N9,_alpha,All
Actual,,,,,,,,,,,,,
N1,29,0,0,0,0,0,0,0,0,0,1,0,30
N10,0,19,1,0,0,0,2,2,0,0,0,6,30
N2,0,0,28,2,0,0,0,0,0,0,0,0,30
N3,0,0,0,0,29,0,0,0,0,0,0,2,31
N4,0,0,0,0,0,27,0,0,0,0,0,5,32
N5,0,1,0,0,0,0,28,1,0,0,0,0,30
N6,0,0,1,0,0,0,1,23,0,0,0,5,30
N7,0,0,0,0,0,0,0,0,29,0,0,1,30
N8,0,0,0,0,0,0,0,0,0,20,0,0,20


In [57]:
order=['fileclass','filetype','purpose','keyword','N1','N2','N3','N4','N5','N6','N7','N8','N9','N10','bias']

kxddf=kxddf[order]
kxddf

In [34]:
order=['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10','_alpha','All']
df_confusionfg=df_confusionfg.reindex(order)
df_confusionfg=df_confusionfg[order].fillna(0)
df_confusionfg

Predicted,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,_alpha,All
Actual,,,,,,,,,,,,
N1,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30.0
N2,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
N3,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,31.0
N4,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,32.0
N5,0.0,0.0,0.0,0.0,28.0,1.0,0.0,0.0,0.0,1.0,0.0,30.0
N6,0.0,1.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,5.0,30.0
N7,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,1.0,30.0
N8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0
N9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,1.0,30.0


In [35]:
ch=pd.DataFrame(columns=fgdf.columns)
for i,r in fgdf.iterrows():
    if (not (r['filegroupclass'] in r['predicted_classes'])) and  (r['group_predicted_class']=="Conflict"):
        
        ch=ch.append(r)
ch

,filegroup,filegroupclass,files,filetypes,predicted_classes,group_predicted_class


In [36]:
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/Keyword_Analysis.xlsx')
kxddf.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/File_Classification_Analysis(AllFiles).xlsx')
ddf.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/ConfusionMatrix(All_Files).xlsx')
df_confusion.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/File_Group_Classification_Analysis.xlsx')
fgdf.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('/home/thrymr/fuzzy2/ConfusionMatrix(FileGroup).xlsx')
df_confusionfg.to_excel(writer,'Sheet1')
writer.save()

In [37]:

with open('Modified_train1.pickle', 'rb') as handle:
    a=pickle.load(handle)
fgdf=a['filegroup']
fdf=a['documents']
kdf=a['keywords']

Encoding Decoding

In [38]:
# kdf[kdf['fileclass']=='N5']
import unicodedata

In [39]:
key=kdf[kdf['fileclass']=='N5'].loc[122]['keyword']
# fdf[(fdf['filename'].str.contains('/01 ')) &(fdf['fileclass']=='N6')&(fdf['filetype']=='NOTIFICATION')]

In [40]:
procedures={'CAM':'Cambiario','COG':'Cognicion','CON':'Concurso Acreedores','EJE':'Juicio Ejecutivo','ETJ':'EJECUCION DE TITULOS JUDICIAL','ETJH':'ETJ Continuacion Hipotecario','ETN':'EJECUCION DE TITULOS NO JUDICIAL',\
            'HIP':'EJECUCION Hipotecaria','MCU':'Menor Cuantia','MON':'Monitorio','ORD':'Ordinario','TDO':'Terceria De Domino','TMD':'Terceria De Mejor Derecho','VER':'Verbal','PEN':'Penal','RAP':'Recurso De Apelacion',\
            'PTC':'Pieza Tasacion Costas','PSO':'Pieza Seperada Oposicion','AUX':'Auxilio Nacional','CNJ':'Cosignacion Judicial','RCS':'Recurso De Casacion','INC':'Incidente Concursal','MC':'Medidas Cautelares','CN':'Conciliacion'}

In [41]:
extKeywords=kdf[kdf['purpose']=='EXTRACTION']

In [42]:
extKeywords['notification_type']=extKeywords['notification_type'].apply(lambda x : x.split('-')[1])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
set(extKeywords['notification_type'].values.tolist())

{u' AMOUNT',
 u' DATE/TIME',
 u' NEGATIVE RESULT OF SEARCH',
 u' RACMO REASON FOR REJECTION',
 u' TIME FRAME',
 u' UNSPECIFIED REJECTED TRANSFER OF LEGAL REPRESENTATION',
 u'RACMO REASON REJECTED CLAIMS '}

In [56]:
# extKeywords[extKeywords['notification_type'].str.contains('TIME FRAME')]
# extKeywords[extKeywords['fileclass']=='N6']

In [45]:
fdf[fdf['filename'].isin(['N5 REJECTED CLAIMS/INADMISION 18-JUST.pdf'])]

,filename,filetype,hasid,table_response,text_response,fileclass,filegroup,keywords
0,N5 REJECTED CLAIMS/INADMISION 18-JUST.pdf,TICKET,353448baec4bed856f04cb31a1573bc0e44b8d34b26b92...,"{""table_1"": {""Destinatarios"": {""_value"": ""ALCO...",mensaje lexnet - notificacion\nmensaje\nidlexn...,N5,INADMISION18,{u'N5': [[u'auto inadmision contrato ilegible']]}


In [46]:
numspan={"1":"uno","2":"dos","3":"tres","4":"cuatro","5":"cinco","6":"seis","7":"siete","8":"ocho","9":"nueve","10":"diez","11":"once","12":"doce","13":"trece","14":"catorce","15":"quince","16":"dieciseis","17" :"diecisiete","18":"dieciocho","19":"diecinueve","20" : "veinte","21" : "veintiuno","22"  :"veintidós","23" : "veintitrés","24" : "veinticuatro","25" : "veinticinco","26":"veintiséis","27" :"veintisiete","28": "veintiocho","29"  :"veintinueve","30" : "treinta"}
numcat={"1":"un","2":"dos","3":"tres","4":"quatre","5":"cinc","6":"sis","7":"set","8":"vuit","9":"nou","10":"deu","11":"onze","12":"dotze","13":"tretze","14":"catorze","15":"quinze","16":"setze","17" :"disset","18":"divuit","19":"dinou","20" : "vint"}

In [53]:
c=0
fgdf['Numlist']=[[] for _ in range(len(fgdf))]
fgdf['MIN DAYS']=''
for fi,fr in fgdf.iterrows():
    auto=""
    match2=set()
    match1=set()
    for i,r in fdf[(fdf['filetype']=='NOTIFICATION')&(fdf['filename'].isin(fr['files']))].iterrows():
        s= r['text_response']
        if len(s.strip())>0:
            s='\n'.join(list(filter(None,s.split('\n')))).lower()

            fgdf.loc[fi,'Court']=s.split('\n')[0]
            if 'procurador' in s.lower():
                fgdf.loc[fi,'Solictor']=list(filter(None,s[s.lower().index('procurador')+10:].split('\n')))[0]


            ptype=""

            for x in procedures.values():

                if x.lower() in ' '.join(s.split()):

                    fgdf.loc[fi,'Procedure_Type']=x
                    ptype=x
                    break
            if ptype!="":
                if fr['filegroup']=='INADMISION25':
                            print ' YESSS'
                if 'procedimiento' in s.lower():
                    match=re.search(r'(\d{1,20}/\d{4})',s.split('procedimiento')[1])

                else:
                    s1=''.join(s.split())
                    match1=set(re.findall(r'(\d{1,20}/\d{4})',s1))
                    match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s1)}

            if not match is None :
                    auto=match.group(0)
            elif not (match1-match2) is None and len((match1-match2))>0 :
                    auto=list(match1-match2)[0]
    if(fr['group_predicted_class']=='N1')or(fr['group_predicted_class']=='N3')or(fr['group_predicted_class']=='N4'):
        for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('AMOUNT'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']==kr['filetype'])].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        tex=' '.join(r['text_response'].split())

                        fgdf.loc[fi,'Amount']=tex[tex.index(k1)+len(k1):].split()[0]
    if((fr['group_predicted_class']=='N9')or(fr['group_predicted_class']=='N10')):
#         for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=' '.join(unidecode.unidecode(r['text_response']).split())
    #           
                nls=[]
                nlc=[]
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    
                    k1='dias'
                    
                    if  k1 in t.lower():
                          
                        nls+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dias' in x ]
                    elif  'dies' in t.lower():
                          
                        nlc+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dies' in x ]
                    else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        min=100
                        ls=[]
                        if len(nls)>0:
                            fgdf.at[fi,'Numlist']=nls
                            ls=nls
                            numbers=numspan
                        elif len(nlc)>0:
                            fgdf.at[fi,'Numlist']=nlc
                            ls=nlc
                            numbers=numcat
                        for num in ls:
                            if num in numbers.values():
                                n=int([x for x in numbers.keys() if numbers[x]==num ][0])
                                if n < min:
                                    min=n
                            elif num in numbers.keys():
                                if int(num) < min:
                                    min=int(num)
                            else:
                                print num
                        if min!=100:
                            fgdf.loc[fi,'MIN DAYS']=min
                        
    if (fr['group_predicted_class']=='N5')|(fr['group_predicted_class']=='N6')|(fr['group_predicted_class']=='N9'):
        

        for ki,kr in extKeywords[~(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        fgdf.loc[fi,fr['group_predicted_class']+'-Extraction']=[dict({kr['notification_type']:kr['keyword']})]
                        if fr['group_predicted_class']=='N5':
                            fgdf.loc[fi,'MIN DAYS']=5
    for i,r in fdf[(fdf['filetype']=='TICKET')&(fdf['filename'].isin(fr['files']))].iterrows():
        
            if r.table_response[:5]!='Error':

                js= json.loads(unidecode.unidecode( json.dumps(json.loads(r.table_response), ensure_ascii=False)))
                table2=json.loads(js['table_2'])
#                 print(r['filename'],js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]],table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]])
                c+=1
                fgdf.loc[fi,"Document date"]=js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]]
                fgdf.loc[fi,"Stamp date"]=table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]]
#                 print js
                
                
                
                fgdf.loc[fi,'Solictor']=js["table_1"]['Destinatarios']['_value']
                fgdf.loc[fi,'Court']=js["table_1"]['Remitente'][[x for x in js["table_1"]['Remitente'].keys() if str(x)[:6].lower()=='organo'][0]]
                auto=""
                try:
                    s=json.loads(unidecode.unidecode(r['table_response']))['table_1']['Datos del mensaje']['Procedimiento destino']

                except Exception as e:
                    s=str( json.loads(r['table_response'])['table_1']['Datos del mensaje'])
        #             print str(e)
                match1=set(re.findall(r'(\d{1,20}/\d{4})',s))
                match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s)}
                
                f=False
                for x in procedures.keys():
                    if x.lower() in  s.lower():
                        f=True
                        proced= procedures[x]
                if f:
                    fgdf.loc[fi,'Procedure_Type']=proced
                if not (match1-match2) is None :
                    auto=list(match1-match2)[0]
                    

    fgdf.loc[fi,'Auto']=auto
            

fgdf

In [48]:
writer = pd.ExcelWriter('/home/thrymr/File_Group_Result.xlsx')
fgdf.to_excel(writer,'Sheet1')
writer.save()